# Interactive Demo for User to perform Building Classification

This demo represents a potential product for a user of our application to be able to classify damage of all buildings in a satellite image using any one of our methods CNN, Edge Detection CNN, Siamese Neural Network, SVM, or KNN. We have the best performance with KNN, but all of them are promising for different use cases.


In [1]:
### Install required packages and Mount Drive

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys
import json
import math

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib
from google.colab.patches import cv2_imshow as show
import random
from collections import Counter


# sci-kit learn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score


#siamese imports
from keras.layers import Input, Dense, InputLayer, Conv2D, MaxPooling2D, UpSampling2D, InputLayer, Concatenate, Flatten, Reshape, Lambda, Embedding, dot, GlobalAveragePooling2D
from keras.models import Model, load_model, Sequential
import tensorflow.keras.backend as K
from keras import metrics
from keras.optimizers import SGD
from sklearn.utils import compute_class_weight

print(tf.__version__)

class Demo:

    def __init__(self):
        self.NAME = None
        self.disaster = None
        self.model = None

    ### User Interfacing Functions:

    # Get user name for path specification
    def get_user_name(self):
        valid_usernames = ["ALAN", "PAT", "ANDY"]

        while True:
            # Asking the user to input their name
            user_name = input("Please enter your name: ").strip().upper()

            # Checking if the entered username is valid
            if user_name in valid_usernames:
                # Greeting the user
                print(f"Hello, {user_name}! Nice to meet you.")
                # Returning the user's name
                self.NAME = user_name
                break  # Exit the loop since a valid username is entered
            else:
                print("Invalid username. Please enter ALAN, PAT, or ANDY.")

    # Choose specific disaster
    def choose_disaster(self):
        print("Disasters:\n")
        print("1. guatemala-volcano")
        print("2. hurricane-florence")
        print("3. hurricane-harvey")
        print("4. hurricane-michael")
        print("5. hurricane-matthew")
        print("6. socal-fire")
        print("7. santa-rosa-wildfire")
        print("8. midwest-flooding")
        print("9. palu-tsunami")
        print("10. mexico-earthquake\n")

        num = int(input("Enter a number to specify the disaster: "))
        while(num < 1 or num > 10):
          num = int(input("\nInput was not valid. Enter a number to specify the disaster: "))

        print("\n\n")

        match num:
          case 1: disaster = 'guatemala-volcano'
          case 2: disaster = 'hurricane-florence'
          case 3: disaster = 'hurricane-harvey'
          case 4: disaster = 'hurricane-michael'
          case 5: disaster = 'hurricane-matthew'
          case 6: disaster = 'socal-fire'
          case 7: disaster = 'santa-rosa-wildfire'
          case 8: disaster = 'midwest-flooding'
          case 9: disaster = 'palu-tsunami'
          case 10: disaster = 'mexico-earthquake'

        self.disaster = disaster
        return disaster

    # Choose model
    def choose_model(self):
        print("Models:\n")
        print("1. CNN")
        print("2. CNN with Edge Detection")
        print("3. Siamese NN")
        print("4. SVM")
        print("5. KNN Classifier")

        num = int(input("Enter a number to specify the model: "))
        while(num < 1 or num > 5):
          num = int(input("\nInput was not valid. Enter a number to specify the model: "))

        print("\n\n")

        self.model = num

        return num

    ### Preprocessing Functions:

    # given json data for a building, this function parses the json data and
    # returns an array of points that outline the polygon mask of the building

    def get_points(self, json_building):
        #split the string
        spaces = json_building['wkt'].split(' ')

        # print(spaces)

        #remove the first element 'POLYGON'
        spaces.pop(0)

        #remove the parenthesis from the first element
        spaces[0] = spaces[0][2::]

        #remove the parethesis of the last element
        spaces[len(spaces)-1] = spaces[len(spaces)-1][:-2]

        #get rid of the commas in the second point
        for i in range(len(spaces)):
          spaces[i] = spaces[i].replace(",","")

        # print(spaces)

        #make them integers and tuples for points
        points = []
        for i in range(0,len(spaces),2):
          points.append((int(float(spaces[i])), int(float(spaces[i+1]))))

        points = np.array(points)

        return points

    # given an array of images and a size, this function adds black to the borders
    # of the images to make all the images have the dimensions (size x size x 3)
    def make_same_dimensions(self, images, size):

        square_dim = size

        # print("square_dim: " + str(square_dim))

        new_images = []
        img_count = 1

        # make all images have same dimensions
        for img in images:

          # print("\nimage " + str(img_count))
          img_count+=1
          #print(str(img_count) + ": " + str(img.shape))

          # print("curr_rows : " + str(curr_rows))
          # print("curr_cols: " + str(curr_cols))

          new_img = img
          curr_rows , curr_cols, _ = new_img.shape


          if curr_rows > square_dim:        #dsize is (width,height) which is -> (cols,rows)
            new_img = cv2.resize(new_img, dsize=(curr_cols,square_dim), interpolation=cv2.INTER_CUBIC)
            curr_rows , curr_cols, _ = new_img.shape
            #print(img_count, "here")

          if curr_cols > square_dim:
            new_img = cv2.resize(new_img, dsize=(square_dim,curr_rows), interpolation=cv2.INTER_CUBIC)
            curr_rows , curr_cols, _ = new_img.shape


          curr_rows , curr_cols, _ = new_img.shape
          # add to cols of image if needed
          if square_dim > curr_cols:
            difference = square_dim - curr_cols

            # print("difference in cols: " + str(difference))

            # creates column array with $rows number of rows
            d = np.array([[[0, 0, 0]] for x in range(curr_rows)])
            # print(d.shape)

            # case where you just add one col to the img
            if difference == 1:
                new_img = np.hstack((new_img, d))
            # case where you add equal amount to left and right of image
            else:
              # add one col to left and right of img
              new_img = np.hstack((new_img, d))
              new_img = np.hstack((d, new_img))

              # add rest of cols to left/right of img
              for i in range(math.floor(difference/2)-1):
                new_img = np.hstack((new_img, d))
                new_img = np.hstack((d, new_img))

              # if difference is odd, have an extra column to add
              if difference%2 == 1:
                new_img = np.hstack((new_img, d))

          # update variables to match updated image
          curr_rows , curr_cols, _ = new_img.shape

          # add to rows of image if needed
          if square_dim > curr_rows:
            difference = square_dim - curr_rows

            # print("difference in rows: " + str(difference))

            # creates row array with $height number of cols
            d = np.array([[[0, 0, 0] for x in range(curr_cols)]])
            # print(d.shape)

            # case where you just add one row to the img
            if difference == 1:
                new_img = np.vstack((new_img, d))
            # case where you add equal amount to top and bottom of image
            else:
              # add one col to left and right of img
              new_img = np.vstack((new_img, d))
              new_img = np.vstack((d, new_img))

              # add rest of cols to left/right of img
              for i in range(math.floor(difference/2)-1):
                new_img = np.vstack((new_img, d))
                new_img = np.vstack((d, new_img))

              # if difference is odd, have an extra column to add
              if difference%2 == 1:
                new_img = np.vstack((new_img, d))

          new_rows, new_cols, _ = new_img.shape

          if new_rows != 150 or new_cols != 150:
            print("old dim: ", img.shape)
            print("new dim:", new_img.shape)

          # print("\nnew_rows: " + str(new_rows))
          # print("new_cols: " + str(new_cols))

          new_images.append(new_img)

        return new_images

    def make_same_dimensions_siamese(self,images, size):

      # square_dim = max(max_rows , max_cols)
      square_dim = size

      # print("square_dim: " + str(square_dim))

      new_images = []
      img_count = 1

      # make all images have same dimensions
      for pair in images:

        # print("\nimage " + str(img_count))
        img_count+=1

        new_img0 = pair[0]
        new_img1 = pair[1]
        curr_rows , curr_cols, _ = new_img0.shape


        if curr_rows > square_dim:        #dsize is (width,height) which is -> (cols,rows)
          new_img0 = cv2.resize(new_img0, dsize=(curr_cols,square_dim), interpolation=cv2.INTER_CUBIC)
          curr_rows , curr_cols, _ = new_img0.shape

          #do the same thing to newimg1
          new_img1 = cv2.resize(new_img1, dsize=(curr_cols,square_dim), interpolation=cv2.INTER_CUBIC)

        if curr_cols > square_dim:
          new_img0 = cv2.resize(new_img0, dsize=(square_dim,curr_rows), interpolation=cv2.INTER_CUBIC)
          curr_rows , curr_cols, _ = new_img0.shape

          new_img1 = cv2.resize(new_img1, dsize=(square_dim,curr_rows), interpolation=cv2.INTER_CUBIC)

        curr_rows , curr_cols, _ = new_img0.shape
        # add to cols of image if needed
        if square_dim > curr_cols:
          difference = square_dim - curr_cols

          # print("difference in cols: " + str(difference))

          # creates column array with $rows number of rows
          d = np.array([[[0, 0, 0]] for x in range(curr_rows)])
          # print(d.shape)

          # case where you just add one col to the img
          if difference == 1:
              new_img0 = np.hstack((new_img0, d))
              new_img1 = np.hstack((new_img1, d))
          # case where you add equal amount to left and right of image
          else:
            # add one col to left and right of img
            new_img0 = np.hstack((new_img0, d))
            new_img0 = np.hstack((d, new_img0))

            new_img1 = np.hstack((new_img1, d))
            new_img1 = np.hstack((d, new_img1))

            # add rest of cols to left/right of img
            for i in range(math.floor(difference/2)-1):
              new_img0 = np.hstack((new_img0, d))
              new_img0 = np.hstack((d, new_img0))

              new_img1 = np.hstack((new_img1, d))
              new_img1 = np.hstack((d, new_img1))


            # if difference is odd, have an extra column to add
            if difference%2 == 1:
              new_img0 = np.hstack((new_img0, d))
              new_img1 = np.hstack((new_img1, d))


        # update variables to match updated image
        curr_rows , curr_cols, _ = new_img0.shape

        # add to rows of image if needed
        if square_dim > curr_rows:
          difference = square_dim - curr_rows

          # print("difference in rows: " + str(difference))

          # creates row array with $height number of cols
          d = np.array([[[0, 0, 0] for x in range(curr_cols)]])
          # print(d.shape)

          # case where you just add one row to the img
          if difference == 1:
              new_img0 = np.vstack((new_img0, d))
              new_img1 = np.vstack((new_img1, d))
          # case where you add equal amount to top and bottom of image
          else:
            # add one col to left and right of img
            new_img0 = np.vstack((new_img0, d))
            new_img0 = np.vstack((d, new_img0))

            new_img1 = np.vstack((new_img1, d))
            new_img1 = np.vstack((d, new_img1))

            # add rest of cols to left/right of img
            for i in range(math.floor(difference/2)-1):
              new_img0 = np.vstack((new_img0, d))
              new_img0 = np.vstack((d, new_img0))

              new_img1 = np.vstack((new_img1, d))
              new_img1 = np.vstack((d, new_img1))


            # if difference is odd, have an extra column to add
            if difference%2 == 1:
              new_img0 = np.vstack((new_img0, d))
              new_img1 = np.vstack((new_img1, d))

        new_rows, new_cols, _ = new_img0.shape

        #if new_rows != 150 or new_cols != 150:
          #print("old dim: ", img[0].shape)
          #print("new dim:", new_img0.shape)

        # print("\nnew_rows: " + str(new_rows))
        # print("new_cols: " + str(new_cols))

        new_images.append([new_img0,new_img1])

      return new_images

    # given the json data of a building and the image the building is from, this
    # function creates a cropped image of just the building from the entire image
    def crop_image(self, building, img):

        points = self.get_points(building)
        # print(points)
        sum_x = 0
        sum_y = 0
        for x,y in points:
          sum_x += x
          sum_y += y

        center_x = round(sum_x/len(points))
        center_y = round(sum_y/len(points))
        # print(center_x)
        # print(center_y)
        # image[ miny:maxy, minx:maxx]
        # show(img_post[center_y-50:center_y+50, center_x-50:center_x+50])

        # source for below code: https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python

        rect = cv2.boundingRect(points)
        x,y,w,h = rect

        croped = img[y:y+h, x:x+w].copy()

        ## (2) make mask
        pts = points - points.min(axis=0)

        mask = np.zeros(croped.shape[:2], np.uint8)
        cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

        ## (3) do bit-op
        dst = cv2.bitwise_and(croped, croped, mask=mask)

        return dst

    # Using json path from username, gather the images from the disaster
    def gather_images_json(self, disaster):

        # Identify user and pull base directory path
        if self.NAME == "PAT":
            base_dir = '/content/drive/MyDrive/Capstone Project/Dataset1/train/images/'
        elif self.NAME == "ALAN":
            base_dir = "/content/drive/MyDrive/SeniorSpring/ENEE439D/Capstone Project/Dataset1/train/images/"
        elif self.NAME == "ANDY":
            base_dir = '/content/drive/MyDrive/Spring 2024/ENEE 439D/Capstone Project/Dataset1/train/images/'
        else:
            raise Exception("Invalid user")

        # Specify disaster name, so training and testing based on disaster subset
        # Select from: guatemala-volcano (16 images), hurricane-florence, hurricane-harvey,
        #              hurricane-michael, hurricane-matthew, socal-fire, santa-rosa-wildfire,
        #              midwest-flooding, palu-tsunami, or mexico-earthquake
        disaster_name = disaster

        count_images = 5

        images_train = []
        json_images_train = []

        i = 1
        while len(images_train) < count_images:
            file_name = f"{disaster_name}_{i:08d}_post_disaster.png"
            img_path = os.path.join(base_dir, file_name)
            label_path = img_path.replace('png', 'json').replace('images', 'labels')

            # If files exist for both image and label, then add it as a train or test image/label
            if os.path.exists(img_path) and os.path.exists(label_path):
                images_train.append(img_path)
                with open(label_path, "rb") as file:
                    data = json.load(file)
                    json_images_train.append(data)

            i += 1

        return images_train, json_images_train

      #for gathering the images for the siamese modeling
    def gather_images_siamese_json(self,disaster):

      if self.NAME == "PAT":
        base_dir = '/content/drive/MyDrive/Capstone Project/Dataset1/train/images/'
      elif self.NAME == "ALAN":
        base_dir = "/content/drive/MyDrive/SeniorSpring/ENEE439D/Capstone Project/Dataset1/train/images/"
      elif self.NAME == "ANDY":
        base_dir = '/content/drive/MyDrive/Spring 2024/ENEE 439D/Capstone Project/Dataset1/train/images/'
      else:
        raise Exception("Invalid user")

      disaster_name = disaster

      curr_count_train = 0
      curr_count_test = 0
      count_train = 11
      count_test = 4
      images_train = []
      json_images_train = []

      images_test = []
      json_images_test = []
      train = True

      for file in os.listdir(base_dir):
        if disaster_name in file and file.endswith("post_disaster.png"):
          if train:
            curr_count_train+=1
          else:
            curr_count_test+=1

          #make the pre and post images a pair:
          img_path = base_dir+file
          pre_img_path = base_dir+file.replace('post_disaster.png',"pre_disaster.png")
          post_img_path = base_dir+file
          pre_post_pair = [pre_img_path,post_img_path]

          if train:
            images_train.append(pre_post_pair)
          else:
            images_test.append(pre_post_pair)


          # get the json data
          label_path = img_path.replace('png', 'json').replace('images', 'labels')
          with open(label_path, "rb") as file:
              data = json.load(file)
          if train:
            json_images_train.append(data)
          else:
            json_images_test.append(data)

          if curr_count_train >= count_train:
            train = False

          if not train and curr_count_test >= count_test:
            break

      return images_train,images_test,json_images_train,json_images_test





    #Used as a reference for function idea: https://www.kaggle.com/code/lezwon/xview2-challenge#kln-47, but chose a different but similar method
    #referene for cv2.fillpoly: https://www.geeksforgeeks.org/draw-a-filled-polygon-using-the-opencv-function-fillpoly/

    # function used to draw labels and output the image with our own dimensions**
    def draw_labels(self, img, data):
      for building in data['features']['xy']:

        #split the string
        spaces = building['wkt'].split(' ')

        #remove the first element 'POLYGON'
        spaces.pop(0)
        #remove the parenthesis from the first element
        spaces[0] = spaces[0][2::]
        #remove the parethesis of the last element
        spaces[len(spaces)-1] = spaces[len(spaces)-1][:-2]

        #get rid of the commas in the second point
        for i in range(len(spaces)):
          spaces[i] = spaces[i].replace(",","")

        #make them integers and tuples for points
        points = []
        for i in range(0,len(spaces),2):
          points.append((int(float(spaces[i])), int(float(spaces[i+1]))))

        points = np.array(points)

        #grab the damage label
        damage_label = building["properties"]["subtype"]

        #CV2 uses BGR not RGB
        #default to cyan
        COLOR = (255,255,25)
        if damage_label == "no-damage":
          #no-damage is white
          COLOR = (255,255,255)
        if damage_label == "minor-damage":
          #minor damage is green
          COLOR = (128,255,102)
        if damage_label == "major-damage":
          #major is orange
          COLOR = (0,153,250)
        if damage_label == "destroyed":
          #red for destoyed
          COLOR = (0,0,255)

        #draw the proper polygon
        cv2.fillPoly(img, [points],COLOR)

      return img

    # Image cropping to specific width and height
    def show_with_size(self, image, width, height, title):
        plt.figure(figsize=(width, height))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')
        plt.show()

    # Show pre and post disaster images
    def show_pre_post(self, images_train, index):
        sample_img = images_train[index]

        # Show pre-disaster image
        img_path_pre = sample_img.replace('post', 'pre')
        img_pre = cv2.imread(img_path_pre)
        # print("Pre disaster Building Damage Assessment Image: ")
        # self.show_with_size(img_pre, width=8, height=6, "Predisaster Image")  # Adjust width and height as needed

        # Show post-disaster image
        img_path_post = img_path_pre.replace('pre', 'post')
        img_post = cv2.imread(img_path_post)
        # print("Post disaster Building Damage Assessment Image: ")
        # self.show_with_size(img_post, width=8, height=6, "Postdisaster Image")  # Adjust width and height as needed

        fig = plt.figure(figsize=(12, 12))
        ax1 = fig.add_subplot(2,2,1)
        ax1.imshow(img_pre)
        ax1.title.set_text('Pre disaster')

        ax2 = fig.add_subplot(2,2,2)
        ax2.imshow(img_post)
        ax2.title.set_text('Post disaster')

     # Draw Labels Onto Image
    def show_post_labels(self, images_train, index):
        sample_img = images_train[index]
        img_post = cv2.imread(sample_img)

        # get the json data
        label_path = sample_img.replace('png', 'json').replace('images', 'labels')
        # takes a while for below code to run so once it runs, work with data variable in new cell
        with open(label_path, "rb") as file:
            data = json.load(file)

        img_true_labels = self.draw_labels(img_post,data)
        self.show_with_size(img_true_labels,6,8, "Ground Truth Postdisaster Image")

    # create train and test data without even damage level split
    def get_train_test_split(self, images_train, json_images_train):

        # Splitting images based on the provided ratio
        train_images = []
        train_labels = []
        test_images = []
        test_labels = []

        ratio = 0.85

        for i in range(len(images_train)):

            img_path = images_train[i]
            data = json_images_train[i]

            img = cv2.imread(img_path)

            for building in data['features']['xy']:
                damage_label = building["properties"]["subtype"]
                if damage_label != "un-classified":
                    cropped_img = self.crop_image(building, img)

                    # Append to train or test based on the split ratio
                    if i != 2 and np.random.uniform(0, 1) < ratio:
                        train_images.append(cropped_img)
                        if damage_label == "no-damage":
                            train_labels.append(0)
                        elif damage_label == "minor-damage":
                            train_labels.append(1)
                        elif damage_label == "major-damage":
                            train_labels.append(2)
                        elif damage_label == "destroyed":
                            train_labels.append(3)
                    else:
                        test_images.append(cropped_img)
                        if damage_label == "no-damage":
                            test_labels.append(0)
                        elif damage_label == "minor-damage":
                            test_labels.append(1)
                        elif damage_label == "major-damage":
                            test_labels.append(2)
                        elif damage_label == "destroyed":
                            test_labels.append(3)

        # Resize images to the same dimensions
        train_images = self.make_same_dimensions(train_images, 150)
        test_images = self.make_same_dimensions(test_images, 150)

        for i in range(0,len(train_images)):
          if train_images[i].shape != (150,150,3):
            print(i,train_images[i].shape)

        train_images = np.array(train_images)
        train_labels = np.array(train_labels)
        test_images = np.array(test_images)
        test_labels = np.array(test_labels)

        return train_images, train_labels, test_images, test_labels

    # Count the occurrences of each value
    def print_class_stats(self, train_or_test_labels):

        # print num images in each class for test images
        counts = Counter(train_or_test_labels)

        # Define label descriptions
        label_descriptions = {
            0: "No damage",
            1: "Minor Damage",
            2: "Major Damage",
            3: "Destroyed"
        }

        # Print the counts with descriptions
        for value, count in counts.items():
            damage = label_descriptions.get(value, "Unclassified")
            print(f"{damage}: {count}")

    ### Model Selection, Training, and Testing

    ## CNN

    # Defines the structure of our CNN. Change this according to our goals
    def build_CNN(self):
          model = keras.Sequential()

          # 128 convolution filters used each of size 3x3
          model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))  #was 150,150,3 making 300,300,3

          # 64 convolution filters used each of size 3x3
          model.add(Conv2D(64, (3, 3), activation='relu'))

          # 32 convolution filters used each of size 3x3
          model.add(Conv2D(32, (3, 3), activation='relu'))

          # choose the best features via pooling
          model.add(MaxPooling2D(pool_size=(2, 2)))

          # randomly turn neurons on and off to improve convergence
          model.add(Dropout(0.25))

          # flatten since too many dimensions, we only want a classification output
          model.add(Flatten())

          # fully connected to get all relevant data
          model.add(Dense(128, activation='relu'))

          # one more dropout
          model.add(Dropout(0.5))

          # output a softmax to squash the matrix into output probabilities
          model.add(Dense(4, activation='softmax'))

          # Before the model is ready for training, it needs a few more settings.
            # Loss function* - measures how accurate the model is during training, we want to minimize this with the optimizer.
            # Optimizer - how the model is updated based on the data it sees and its loss function.
            # Metrics - used to monitor the training and testing steps. "accuracy" is the fraction of images that are correctly classified.

          opt = tf.keras.optimizers.Adam()
          # opt = tf.keras.optimizers.SGD()
          # opt = tf.keras.optimizers.Adagrad()
          # opt = tf.keras.optimizers.Nadam()

          # loss_fn = keras.losses.Huber()
          # loss_fn = keras.losses.MeanAbsoluteError()
          # loss_fn = keras.losses.LogCosh()
          loss_fn = 'sparse_categorical_crossentropy'

          model.compile(optimizer=opt,
                        loss=loss_fn,
                        metrics=['accuracy'])

          return model

    # CNN Training Steps

      # 1. Feed the training data to the model: In this example, our training data are the `train_images` and `train_labels` arrays, which correspond to the building images and their postdisaster labels.
      # 2. The model learns to associate images and labels.
      # 3. We ask the model to make predictions about the `test_images` array. We verify that the predictions match the labels from the `test_labels` array. By doing this, we can evaluate the accuracy of our model on predicting damage labels for post-disaster building test images.

      # To start training,  call the `model.fit` method—the model is "fit" to the training data:

    # Training of CNN Model
    def cnn_train(self, model, train_images, train_labels):

          history = model.fit(train_images, train_labels, epochs=10,shuffle = True, batch_size = 5)

    # Testing of CNN Model
    def cnn_test(self, model, test_images, test_labels):

          # Evaluate Model Accuracy
          test_loss, test_acc = model.evaluate(test_images, test_labels)

          print('Test accuracy:', test_acc)

    ## SVM

    # Perform PCA on training and testing datasets
    def svm_pca(self, train_images, test_images):
          # flatten data so its not 150x150x3
          train_i = []

          for sample in train_images:
            train_i.append(sample.flatten())

          test_i = []

          for sample in test_images:
            test_i.append(sample.flatten())


          # pca = PCA(n_components=pca_components)  # Set the number of components you want

          # # Fit and transform the data
          # train_i_pca = pca.fit_transform(train_i)
          # test_i_pca = pca.fit_transform(test_i)

          # return train_i_pca, test_i_pca

          return train_i, test_i

    # Train and Test for SVM Classifier
    def svm_train_test(self, train_images, train_labels, test_images, test_labels):

          # train the classifier
          poly = svm.SVC(kernel='poly', degree=3, C=1).fit(train_images, train_labels)
          poly_pred = poly.predict(test_images)

          # calculate the accuracy and f1 scores for SVM with Polynomial kernel
          poly_accuracy = accuracy_score(test_labels, poly_pred)
          poly_f1 = f1_score(test_labels, poly_pred, average='weighted')
          print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
          print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

          return poly

    ## CNN Edge Detection

  #for converting an image to its edges
    def convert_to_edge(self,building_img, t = 'Sobel'):
          #make the image black and white
          #first change to uint8 since thats whats expected here
          new_img = np.array(building_img, dtype=np.uint8)

          ##TRY some filtering of some kind
          kernel_outline = np.array([[-1 ,-1, -1], [-1, 8, -1], [-1, -1, -1]])
          kernel_sharpen = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
          new_img = cv2.filter2D(new_img, -1, kernel_sharpen)
          #new_img = cv2.filter2D(new_img, -1, kernel_outline)


          #make black and white
          new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)

          #new_img = cv2.GaussianBlur(new_img,(5,5),cv2.BORDER_DEFAULT)


          #do the edge detection depending on the param
          if t == 'Sobel':
            new_img = cv2.Sobel(src=new_img, ddepth=cv2.CV_64F, dx=2, dy=2, ksize=3)
          else:
            new_img = cv2.Canny(image=new_img, threshold1=100, threshold2=1000)

          return new_img


    ## Siamese
    def make_siamese_pairs(self,images_train,images_test,json_images_train,json_images_test):

          train_images = []
          train_labels = []

          for i in range(len(images_train)):

            #pre and post labels
            img_pre_path = images_train[i][0]
            img_post_path = images_train[i][1]

            img_path = images_train[i]
            data = json_images_train[i]

            #pre and post
            img_pre = cv2.imread(img_pre_path)
            img_post = cv2.imread(img_post_path)

            for building in data['features']['xy']:
              damage_label = building["properties"]["subtype"]
              if damage_label != "un-classified":
                cropped_img_pre = self.crop_image(building, img_pre)
                cropped_img_post = self.crop_image(building,img_post)
                #buildings_train.append(cropped_img)
                train_images.append([cropped_img_pre,cropped_img_post])


                if damage_label == "no-damage":
                  train_labels.append(0)
                if damage_label == "minor-damage":
                  train_labels.append(1)
                if damage_label == "major-damage":
                  train_labels.append(2)
                if damage_label == "destroyed":
                  train_labels.append(3)

          #print(train_images)
          train_images = self.make_same_dimensions_siamese(train_images, 64) #was 150, making 300#make_same_dimensions(buildings_train, 300) #was 150, making 300

          test_images = []
          test_labels = []

          for i in range(len(images_test)):

            #pre and post labels
            img_pre_path = images_train[i][0]
            img_post_path = images_train[i][1]


            img_path = images_test[i]
            data = json_images_test[i]

            #pre and post
            img_pre = cv2.imread(img_pre_path)
            img_post = cv2.imread(img_post_path)

            for building in data['features']['xy']:
              damage_label = building["properties"]["subtype"]
              if damage_label != "un-classified":
                cropped_img_pre = self.crop_image(building, img_pre)
                cropped_img_post = self.crop_image(building,img_post)
                #buildings_train.append(cropped_img)
                test_images.append([cropped_img_pre,cropped_img_post])


                if damage_label == "no-damage":
                  test_labels.append(0)
                if damage_label == "minor-damage":
                  test_labels.append(1)
                if damage_label == "major-damage":
                  test_labels.append(2)
                if damage_label == "destroyed":
                  test_labels.append(3)

          test_images = self.make_same_dimensions_siamese(test_images, 64) #make_same_dimensions(buildings_test, 300)#was 150 making 300

          #then convert the pairs into their binary category

          no_damage_pairs_train = []
          some_damage_pairs_train = []

          no_damage_pairs_test = []
          some_damage_pairs_test = []

          for i in range(len(train_labels)):
            if train_labels[i] == 0:
              no_damage_pairs_train.append(train_images[i])
            else:
              some_damage_pairs_train.append(train_images[i])

          for i in range(len(test_labels)):
            if train_labels[i] == 0:
              no_damage_pairs_test.append(test_images[i])
            else:
              some_damage_pairs_test.append(test_images[i])


          #finally make it ready for input
          #seperate the pre and post and new labels
          #print(len(no_damage_pairs_train))
          #print(len(some_damage_pairs_train))
          #print(train_labels)
          #print(test_labels)


          pre = []
          post = []
          labels = []
          #no damage labels
          for i in range(len(no_damage_pairs_train)):
            pre.append(no_damage_pairs_train[i][0])
            post.append(no_damage_pairs_train[i][1])
            labels.append(1)
          #some damage labels
          for i in range(len(some_damage_pairs_train)):
            pre.append(some_damage_pairs_train[i][0])
            post.append(some_damage_pairs_train[i][1])
            labels.append(0)


          #convert to np arrays
          pre = np.array(pre)
          post = np.array(post)
          labels = np.array(labels)


          #get the testing data set up
          #seperate the pre and post and new labels
          pre_test = []
          post_test = []
          labels_test = []
          #no damage labels
          for i in range(len(no_damage_pairs_test)):
            pre_test.append(no_damage_pairs_test[i][0])
            post_test.append(no_damage_pairs_test[i][1])
            labels_test.append(1)
          #some damage labels
          for i in range(len(some_damage_pairs_test)):
            pre_test.append(some_damage_pairs_test[i][0])
            post_test.append(some_damage_pairs_test[i][1])
            labels_test.append(0)


          #convert to np arrays
          pre_test = np.array(pre_test)
          post_test = np.array(post_test)
          labels_test = np.array(labels_test)

          return pre,post,labels,pre_test,post_test,labels_test

    #modeling
    def build_siamese_model(self):
          #sister CNNs
          input_layer = Input((64, 64, 3))
          layer1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
          layer2 = MaxPooling2D((2, 2), padding='same')(layer1)
          layer3 = Conv2D(128, (3, 3), activation='relu', padding='same')(layer2)
          layer4 = MaxPooling2D((2, 2), padding='same')(layer3)
          layer5 = Conv2D(256, (3, 3), activation='relu', padding='same')(layer4)
          layer6 = MaxPooling2D((2, 2), padding='same')(layer5)
          layer7 = Conv2D(512, (3, 3), activation='relu', padding='same')(layer6)
          layer8 = MaxPooling2D((2, 2), padding='same')(layer7)
          layer9 = Dropout(0.25)(layer8)
          layer10 = Flatten()(layer9)
          layer11 = Dense(1024,activation = 'relu')(layer10)
          layer12 = Dropout(0.25)(layer11)
          #layer9 = GlobalAveragePooling2D()(layer8)
          layer13 = Dense(144)(layer12)
          #layer14 = Dense(64)(layer13)

          # Create model
          base_model = Model(inputs=input_layer, outputs=layer13)

          #create the siamese modeling:
          input_pre = Input((64,64,3))
          input_post = Input((64,64,3))

          pre_model = base_model(input_pre)
          post_model = base_model(input_post)

          #rather than dot layer, this source reccomends a euclidian distance layer:
          #https://pyimagesearch.com/2020/11/30/siamese-networks-with-keras-tensorflow-and-deep-learning/


          def euclidean_distance(vectors):

              # unpack the vectors into separate lists
              (featsA, featsB) = vectors
              # compute the sum of squared distances between the vectors
              sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
              # return the euclidean distance between the vectors
              return K.sqrt(K.maximum(sumSquared, K.epsilon()))

          #contrastive loss layers
          distance = Lambda(euclidean_distance)([pre_model, post_model])
          output = Dense(1, activation="sigmoid")(distance)

          siamese_model = Model(inputs=[input_pre, input_post], outputs=output)
          return siamese_model

    ## KNN

    def knn_train_test(self, train_images, train_labels, test_images, test_labels):

          # Flatten the images
          train_images_flat = [img.flatten() for img in train_images]
          test_images_flat = [img.flatten() for img in test_images]

          # Define a range of neighbors
          neighbors = range(1, 11)

          accuracies = []
          f1_scores = []

          for k in neighbors:
              # Initialize the K-NN classifier
              knn_classifier = KNeighborsClassifier(n_neighbors=k)

              # Train the classifier
              knn_classifier.fit(train_images_flat, train_labels)

              # Predict the labels for the test images
              predicted_labels = knn_classifier.predict(test_images_flat)

              # Calculate accuracy
              accuracy = accuracy_score(test_labels, predicted_labels)
              accuracies.append(accuracy)

              # Calculate F1 score
              f1 = f1_score(test_labels, predicted_labels, average='weighted')
              f1_scores.append(f1)

          # Plotting
          plt.plot(neighbors, accuracies, marker='o', label='Accuracy')
          plt.plot(neighbors, f1_scores, marker='+', label='F1 Score')
          plt.title('Accuracy and F1 Score vs. Number of Neighbors')
          plt.xlabel('Number of Neighbors')
          plt.ylabel('Score')
          plt.legend()
          plt.xticks(neighbors)

          # Create DataFrame
          df = pd.DataFrame({'Accuracies': accuracies, 'F1 Scores': f1_scores, '# of neighbors': neighbors})

          # Print DataFrame
          print(df)

    ### Postprocessing: Classification Results Analysis

    ## CNN

    # get confustion matrix for CNN
    def get_matrix_cnn(self, model, images_train, json_images_train, index):

          test_predictions = []

          # for img in test_images:
          #   p = model.predict(img)
          #   test_predictions.append(p)

          img_path = images_train[index]
          data = json_images_train[index]

          img = cv2.imread(img_path)

          test_data_demo = []

          for building in data['features']['xy']:
              damage_label = building["properties"]["subtype"]
              if damage_label != "un-classified":
                  cropped_img = self.crop_image(building, img)
                  if damage_label == "no-damage":
                      test_data_demo.append((cropped_img, 0))
                  elif damage_label == "minor-damage":
                      test_data_demo.append((cropped_img, 1))
                  elif damage_label == "major-damage":
                      test_data_demo.append((cropped_img, 2))
                  elif damage_label == "destroyed":
                      test_data_demo.append((cropped_img, 3))

          test_images_demo, test_labels_demo = zip(*test_data_demo)

          # Resize images to the same dimensions
          test_images_demo = self.make_same_dimensions(test_images_demo, 150)

          test_images_demo = np.array(test_images_demo)
          test_labels_demo = np.array(test_labels_demo)

          test_predictions = model.predict(test_images_demo)

          preds = []
          for probabilities in test_predictions:
            maximum = max(probabilities[0], probabilities[1], probabilities[2], probabilities[3])
            if maximum == probabilities[0]:
              preds.append(0)
            elif maximum == probabilities[1]:
              preds.append(1)
            elif maximum == probabilities[2]:
              preds.append(2)
            else:
              preds.append(3)

          # actual values are the rows
          # predicted values are the cols
          #we had this backwards, but i confirmed with counts of truth labels
          matrix = tf.math.confusion_matrix(test_labels_demo, preds)

          return matrix, preds

    ## SVM

    # Get confusion matrix for SVM
    def get_matrix_svm(self, poly, images_train, json_images_train, index):

          test_predictions = []

          img_path = images_train[index]
          data = json_images_train[index]

          img = cv2.imread(img_path)

          test_data_demo = []

          for building in data['features']['xy']:
              damage_label = building["properties"]["subtype"]
              if damage_label != "un-classified":
                  cropped_img = self.crop_image(building, img)
                  if damage_label == "no-damage":
                      test_data_demo.append((cropped_img, 0))
                  elif damage_label == "minor-damage":
                      test_data_demo.append((cropped_img, 1))
                  elif damage_label == "major-damage":
                      test_data_demo.append((cropped_img, 2))
                  elif damage_label == "destroyed":
                      test_data_demo.append((cropped_img, 3))

          test_images_demo, test_labels_demo = zip(*test_data_demo)

          # Resize images to the same dimensions
          test_images_demo = self.make_same_dimensions(test_images_demo, 150)

          test_images_demo = np.array(test_images_demo)
          test_labels_demo = np.array(test_labels_demo)

          # print("test_images_demo size: " + str(test_images_demo.shape) + "\n")

          test_i = []

          for sample in test_images_demo:
            test_i.append(sample.flatten())

          # print("test_i size: " + str(len(test_i)) + "\n")

          # pca = PCA(n_components=pca_components)  # Set the number of components you want

          # # Fit and transform the data
          # test_i_pca = pca.fit_transform(test_i)

          # poly_pred = poly.predict(test_i_pca)

          poly_pred = poly.predict(test_i)

          matrix_poly = tf.math.confusion_matrix(test_labels_demo, poly_pred)

          return matrix_poly, poly_pred


    ## CNN Edge Detection

    ## Siamese

    ## KNN

    def get_matrix_knn(self, train_images, train_labels, test_images, test_labels):

          # Flatten the images
          train_images_flat = [img.flatten() for img in train_images]
          test_images_flat = [img.flatten() for img in test_images]

          # Specify number of neighbors k
          while True:
            try:
                # Specify number of neighbors k
                k = int(input("Enter the number of neighbors (between 1 and 10) for a Confusion Matrix of predictions: "))
                if k < 1 or k > 10:
                    raise ValueError("Please enter a number between 1 and 10.")
                else:
                    break
            except ValueError as e:
                print(e)

          # Initialize the K-NN classifier
          knn_classifier = KNeighborsClassifier(k)

          # Train the classifier
          knn_classifier.fit(train_images_flat, train_labels)

          # Predict the labels for the test images
          test_predictions = knn_classifier.predict(test_images_flat)

          # actual values are the rows
          # predicted values are the columns
          matrix = tf.math.confusion_matrix(test_labels, test_predictions)

          return matrix, test_predictions

    ##Code to generate a nice looking confusion matrix by color
    #utalizes the matplotlib pcolormesh, I followed documentation on matplotlib.org for various sections of this code
    #links:
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html
    #https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py
    #https://matplotlib.org/stable/gallery/text_labels_and_annotations/placing_text_boxes.html

    def visualize_confusion_matrix(self, matrix, train_images, test_images):
          #build the data array, flipping the order of the rows
          #this is nessassary since the pcolormesh has 0 at the bottom rather than top like a typical
          #confusion matrix
          data = []
          for row in range(0,4):
            currCol = []
            for col in range(0,4):
              currCol.append(matrix[row][col].numpy())
            data.insert(0,currCol)

          print(data[0])
          #use the pcolormesh function, following the documentations
          Z = data
          x = np.arange(0,4,1)
          y = np.arange(0,4,1)

          maxVal = max(max(data[0]),max(data[1]),max(data[2]),max(data[3]))

          fig, ax = plt.subplots()
          ax.pcolormesh(x, y, Z, cmap = 'viridis')

          #calculate percentages of the columns for annotation
          percentages = []
          for col in range(0,4):
            currCol = []
            sum = 0
            #gets the total guesses
            for i in range(0,4):
              sum = sum + data[i][col]
            #calculates percentages
            for i in range(0,4):
              if sum != 0:
                currCol.append((data[i][col]/sum) * 100)
              else:
                currCol.append(0.00)
            percentages.append(currCol)

          #adding text annotations in boxes
          for i in range(0,4):
            for j in range(0,4):
              #change the text color if nessassary, 75% of max make the text black
              if data[i][j] < maxVal*0.75:
                text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="w")
              else:
                text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="k")

          #add additional text to bottom corner
          ax.text(-1.5,-1.05, "Perfect Classifier = Only Diagonal Filled")
          ax.text(-1.5,-1.25, "% = Correctness given guess type")
          ax.text(-1.5,-1.45, "# of Train Buildings: " + str(train_images.shape[0]))
          ax.text(-1.5,-1.65, "# of Test Buildings: " + str(test_images.shape[0]))


          #axes info
          classLabelsX = ["No Damage", 'Minor Damage', 'Major Damage', 'Destroyed']
          classLabelsY = ['Destroyed','Major Damage','Minor Damage','No Damage']
          ax.set_xticks(np.arange(4), labels=classLabelsX)
          ax.set_yticks(np.arange(4), labels=classLabelsY)

          #plots the confusion matrix with titles
          plt.title("Confusion Matrix")
          plt.xlabel("Predicted Values")
          plt.ylabel("Ground Truth")
          plt.show()


    def visualize_confusion_matrix_siamese(self,matrix):
      #build the data array, flipping the order of the rows
      #this is nessassary since the pcolormesh has 0 at the bottom rather than top like a typical
      #confusion matrix
      data = []
      for row in range(0,2):
        currCol = []
        for col in range(0,2):
          currCol.append(matrix[row][col].numpy())
        data.insert(0,currCol)

      print(data[0])
      #use the pcolormesh function, following the documentations
      Z = data
      x = np.arange(0,2,1)
      y = np.arange(0,2,1)

      maxVal = max(max(data[0]),max(data[1]))

      fig, ax = plt.subplots()
      ax.pcolormesh(x, y, Z, cmap = 'viridis')

      #calculate percentages of the columns for annotation
      percentages = []
      for col in range(0,2):
        currCol = []
        sum = 0
        #gets the total guesses
        for i in range(0,2):
          sum = sum + data[i][col]
        #calculates percentages
        for i in range(0,2):
          if sum != 0:
            currCol.append((data[i][col]/sum) * 100)
          else:
            currCol.append(0.00)
        percentages.append(currCol)

      #adding text annotations in boxes
      for i in range(0,2):
        for j in range(0,2):
          #change the text color if nessassary, 75% of max make the text black
          if data[i][j] < maxVal*0.75:
            text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="w")
          else:
            text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="k")

      #add additional text to bottom corner
      ax.text(-1.5,-1.05, "Perfect Classifier = Only Diagonal Filled")
      ax.text(-1.5,-1.25, "% = Correctness given guess type")
      #ax.text(-1.5,-1.45, "# of Train Buildings: " + str(train_images.shape[0]))
      #ax.text(-1.5,-1.65, "# of Test Buildings: " + str(test_images.shape[0]))


      #axes info
      classLabelsX = ["Damage","No Damage"]
      classLabelsY = ["No Damage", "Damage"]
      ax.set_xticks(np.arange(2), labels=classLabelsX)
      ax.set_yticks(np.arange(2), labels=classLabelsY)

      #plots the confusion matrix with titles
      plt.title("Confusion Matrix")
      plt.xlabel("Predicted Values")
      plt.ylabel("Ground Truth")
      plt.show()


    def visualize_confusion_matrix_svm(self, matrix, train_images, test_images):
          #build the data array, flipping the order of the rows
          #this is nessassary since the pcolormesh has 0 at the bottom rather than top like a typical
          #confusion matrix
          data = []
          for row in range(0,4):
            currCol = []
            for col in range(0,4):
              currCol.append(matrix[row][col].numpy())
            data.insert(0,currCol)

          print(data[0])
          #use the pcolormesh function, following the documentations
          Z = data
          x = np.arange(0,4,1)
          y = np.arange(0,4,1)

          maxVal = max(max(data[0]),max(data[1]),max(data[2]),max(data[3]))

          fig, ax = plt.subplots()
          ax.pcolormesh(x, y, Z, cmap = 'viridis')

          #calculate percentages of the columns for annotation
          percentages = []
          for col in range(0,4):
            currCol = []
            sum = 0
            #gets the total guesses
            for i in range(0,4):
              sum = sum + data[i][col]
            #calculates percentages
            for i in range(0,4):
              if sum != 0:
                currCol.append((data[i][col]/sum) * 100)
              else:
                currCol.append(0.00)
            percentages.append(currCol)

          #adding text annotations in boxes
          for i in range(0,4):
            for j in range(0,4):
              #change the text color if nessassary, 75% of max make the text black
              if data[i][j] < maxVal*0.75:
                text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="w")
              else:
                text = ax.text(j, i, str(data[i][j]) + "\n" +str(round(percentages[j][i],2)) + "%" , ha="center", va="center", color="k")

          #add additional text to bottom corner
          ax.text(-1.5,-1.05, "Perfect Classifier = Only Diagonal Filled")
          ax.text(-1.5,-1.25, "% = Correctness given guess type")
          ax.text(-1.5,-1.45, "# of Train Buildings: " + str(len(train_images)))
          ax.text(-1.5,-1.65, "# of Test Buildings: " + str(len(test_images)))


          #axes info
          classLabelsX = ["No Damage", 'Minor Damage', 'Major Damage', 'Destroyed']
          classLabelsY = ['Destroyed','Major Damage','Minor Damage','No Damage']
          ax.set_xticks(np.arange(4), labels=classLabelsX)
          ax.set_yticks(np.arange(4), labels=classLabelsY)

          #plots the confusion matrix with titles
          plt.title("Confusion Matrix")
          plt.xlabel("Predicted Values")
          plt.ylabel("Ground Truth")
          plt.show()

    #Used as a reference for function idea: https://www.kaggle.com/code/lezwon/xview2-challenge#kln-47, but chose a different but similar method
    #referene for cv2.fillpoly: https://www.geeksforgeeks.org/draw-a-filled-polygon-using-the-opencv-function-fillpoly/

    # Function to draw predicted labels onto image
    def draw_new_labels(self, img, data, preds):
          index = 0
          for building in data['features']['xy']:

            #split the string
            spaces = building['wkt'].split(' ')

            #remove the first element 'POLYGON'
            spaces.pop(0)
            #remove the parenthesis from the first element
            spaces[0] = spaces[0][2::]
            #remove the parethesis of the last element
            spaces[len(spaces)-1] = spaces[len(spaces)-1][:-2]

            #get rid of the commas in the second point
            for i in range(len(spaces)):
              spaces[i] = spaces[i].replace(",","")

            #make them integers and tuples for points
            points = []
            for i in range(0,len(spaces),2):
              points.append((int(float(spaces[i])), int(float(spaces[i+1]))))

            points = np.array(points)

            #grab the damage label
            # damage_label = building["properties"]["subtype"]

            damage_label = preds[index]

            #CV2 uses BGR not RGB
            #default to cyan
            COLOR = (255,255,25)
            if damage_label == 0:
              #no-damage is white
              COLOR = (255,255,255)
            if damage_label == 1:
              #minor damage is green
              COLOR = (128,255,102)
            if damage_label == 2:
              #major is orange
              COLOR = (0,153,250)
            if damage_label == 3:
              #red for destoyed
              COLOR = (0,0,255)

            #draw the proper polygon
            cv2.fillPoly(img, [points],COLOR)

            index += 1

          return img

    # Show post-disaster image with true labels and with predicted labels
    def show_post_true_predicted(self, images_train, index, preds):
          sample_img = images_train[index]
          img_post = cv2.imread(sample_img)

          # get the json data
          label_path = sample_img.replace('png', 'json').replace('images', 'labels')
          # takes a while for below code to run so once it runs, work with data variable in new cell
          with open(label_path, "rb") as file:
              data = json.load(file)

          img = self.draw_labels(img_post, data)
          self.show_with_size(img,5,8, 'Post-Disaster Image with True Labels')

          img = self.draw_new_labels(img_post, data, preds)
          self.show_with_size(img,5,8, 'Post-Disaster Image with Predicted Labels')


    # main function coordinates user interface in demo
    def main(self):

        # Get user name for path identification
        self.get_user_name()

        # Get disaster type
        disaster = self.choose_disaster()

        # Gather images from disaster
        images_train, json_images_train = self.gather_images_json(disaster)

        # choosing which image in images_train to make the sample image
        index = 2

        # Print pre- and post-disaster images for a sample image from this disaster
        self.show_pre_post(images_train, index)

        # Print post-disaster sample image with its true labels
        self.show_post_labels(images_train, index)

        # Train disaster and test on validation dataset of images
        train_images, train_labels, test_images, test_labels = self.get_train_test_split(images_train, json_images_train)

        print("\nTrain Set Stats")
        self.print_class_stats(train_labels)
        print("\nTest Set Stats")
        self.print_class_stats(test_labels)

        print("\n\n")

        # Choose model and build it accordingly
        model_type = self.choose_model()

        # CNN Model
        if model_type == 1:
            model = self.build_CNN()

            print("Training the CNN")
            self.cnn_train(model, train_images, train_labels)
            print("\nTesting the CNN")
            self.cnn_test(model, test_images, test_labels)

            matrix, preds = self.get_matrix_cnn(model, images_train, json_images_train, index)

        # CNN Edge Detection
        elif model_type == 2:

          #processing the images into edges
          #iterate though the test and the train set doing the edge detection
          edge_train = []
          edge_test = []
          DETECTION_TYPE = "Canny"
          for i in range(0,len(train_images)):
            img_post = self.convert_to_edge(train_images[i], t = DETECTION_TYPE)
            edge_train.append(img_post)
          for i in range(0,len(test_images)):
            img_post = self.convert_to_edge(test_images[i], t = DETECTION_TYPE)
            edge_test.append(img_post)

          #use the same CNN as above
          model = self.build_CNN()

          print("Training the CNN-Edge Detection")
          self.cnn_train(model, train_images, train_labels)
          print("\nTesting the CNN- Edge Detection")
          self.cnn_test(model, test_images, test_labels)

          matrix, preds = self.get_matrix_cnn(model, images_train, json_images_train, index)


        # Siamese Model
        elif model_type == 3:

          #for siamese we need the pre and the post images, so re-gather the pre images
          train_images,test_images,json_images_train,json_images_test = self.gather_images_siamese_json(disaster)

          #print(train_images)
          #print(json_images_train)

          #build the pairs
          pre,post,labels,pre_test,post_test,labels_test = self.make_siamese_pairs(train_images,test_images,json_images_train,json_images_test)

          #build the model
          model = self.build_siamese_model()
          #compile the model
          model.compile(optimizer='adam', loss= 'binary_crossentropy',metrics = ['accuracy'])
          #train the model

          classWeight = compute_class_weight('balanced', classes = np.unique(labels), y = labels)
          classWeight = dict(enumerate(classWeight))

          #model train
          model.fit([pre, post], labels, epochs=10,batch_size = 16,shuffle=True, verbose=True, class_weight = classWeight)

          #predictions
          model_predictions = model.predict([pre_test,post_test])

          #generate the predictions
          preds = []
          for i in range(len(model_predictions)):
            #maximum = max(model_predictions[i][0],model_predictions[i][1])
            #if maximum == model_predictions[i][0]:
            if model_predictions[i] < 0.5:
              preds.append(0)
            else:
              preds.append(1)
          #getting the confusion matrix
          matrix = tf.math.confusion_matrix(labels_test,preds)

        # SVM Model
        elif model_type == 4:
            train_images, test_images = self.svm_pca(train_images, test_images)

            model = self.svm_train_test(train_images, train_labels, test_images, test_labels)

            matrix, preds = self.get_matrix_svm(model, images_train, json_images_train, index)

            print(matrix)

        # KNN Model
        elif model_type == 5:
            self.knn_train_test(train_images, train_labels, test_images, test_labels)
            matrix, preds = self.get_matrix_knn(train_images, train_labels, test_images, test_labels)

        print("\n\nConfusion Matrix for Model Against Sample Image")

        # Post processing to output confusion matrix and postdisaster ground truth and prediction labels
        if model_type != 3 and model_type != 4:
          self.visualize_confusion_matrix(matrix, train_images, test_images)
        elif model_type == 4:
          self.visualize_confusion_matrix_svm(matrix, train_images, test_images)
        else:
          self.visualize_confusion_matrix_siamese(matrix)


        self.show_post_true_predicted(images_train, index, preds)

if __name__ == "__main__":
    demo = Demo()
    demo.main()


MessageError: Error: credential propagation was unsuccessful